# Long Notebook
-   Beatrice Re 104972
-   Claudio Di Salvo 105677
-   Raymond Li 105281

Group Name : 10 TimeBenders

# Analysis
We have done the same analysis on the other location, we omitted them for readability of the notebook.


Attention, this notebook is only partially runnable

## 1. Import Packages

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
import seaborn as sns
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
import xgboost as xgb

## 2. Data Reading and intuitive analysis

In [ ]:
def readData(location):
  if(location == 1):
    y_train_obs_A = pd.read_parquet('data/A/train_targets.parquet')
    X_test_est_A = pd.read_parquet('data/A/X_test_estimated.parquet')
    X_train_est_A = pd.read_parquet('data/A/X_train_estimated.parquet')
    X_train_obs_A = pd.read_parquet('data/A/X_train_observed.parquet')
    return y_train_obs_A, X_test_est_A, X_train_est_A, X_train_obs_A
  elif(location == 2):
    y_train_obs_B = pd.read_parquet('data/B/train_targets.parquet')
    X_test_est_B = pd.read_parquet('data/B/X_test_estimated.parquet' )
    X_train_est_B = pd.read_parquet('data/B/X_train_estimated.parquet')
    X_train_obs_B = pd.read_parquet('data/B/X_train_observed.parquet')
    return y_train_obs_B, X_test_est_B , X_train_est_B, X_train_obs_B
  else:
    y_train_obs_C = pd.read_parquet('data/C/train_targets.parquet')
    X_test_est_C = pd.read_parquet('data/C/X_test_estimated.parquet')
    X_train_est_C = pd.read_parquet('data/C/X_train_estimated.parquet')
    X_train_obs_C = pd.read_parquet('data/C/X_train_observed.parquet')
    return y_train_obs_C, X_test_est_C, X_train_est_C, X_train_obs_C

In [ ]:
df_train_est_A, df_train_obs_A, df_test_A, target = readData(1)

The date_forecast must be the index since we are predicting the future

In [ ]:
df_train_est_A.set_index('date_forecast', inplace=True)
df_train_obs_A.set_index('date_forecast', inplace=True)
df_test_A.set_index('date_forecast', inplace=True)
target.set_index('time',inplace = True)

In [ ]:
df_train = pd.concat([df_train_obs_A,df_train_est_A])

### 2.1. Visual Analysis

We decided to scale the data to have a proper way to visualize it

In [ ]:
def scale_df(df):
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
    return df_scaled

In [ ]:
df = df_train
df_target = target

df.index = pd.to_datetime(df.index)
df_target.index = pd.to_datetime(df_target.index)

df_scaled = scale_df(df)
# Ensuring that df_target contains only numeric columns
df_target_numeric = df_target.select_dtypes(include=[np.number])
df_target_scaled = scale_df(df_target_numeric)

# Number of features to plot
num_features = len(df.columns)

# Creating a figure
plt.figure(figsize=(15, 5*num_features))

for i, col in enumerate(df.columns, 1):
    plt.subplot(num_features, 1, i)

    plt.plot( df_scaled[col], label=f'Scaled Feature: {col}', linestyle='-', marker='o')
    plt.plot( df_target_scaled['pv_measurement'], label='Scaled PV Measurement', linestyle='-', marker='o')

    plt.xlabel('Time')
    plt.ylabel('Scaled Values')
    plt.title(f'Scaled {col} and PV Measurement over Time')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

The data visualization clearly indicated that pv_measurement values were higher in summer, which could be attributed to the more intense solar radiation resulting from cloudless skies and longer daylight hours.

## 3. EDA

### 3.1 Impute the NaN

In [ ]:
df_train.isnull().mean()

We encountered a few columns with a high percentage of missing values (NaNs). Consequently, we chose to drop two columns ('snow_density:kgm3', 'elevation:m') and impute the remaining ones.

In [ ]:
df_train.drop[['snow_density:kgm3', 'elevation:m']]

To address the missing values, we experimented with various imputation methods. Our initial approach involved using a neighbor-based technique.

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=24)
df_train_est_A_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

This was too much expensive in computational POV. So we stepped over.

To determine the most effective imputation method, we established an evaluation protocol based on LightGBM. However, none of the imputation solutions we tested in this evaluation were ultimately utilized.

In [ ]:
df_nandrop = df
df_nandrop = df_nandrop.drop(columns= ['snow_densitykgm3'],axis = 1)
df_nandrop = df_nandrop.dropna()

df_nan0 = df.fillna(0)
df_nanmedian = df.fillna(df.median())

In [ ]:
df_dict = {
    'drop' : df_nandrop,
    'zero' : df_nan0,
    'median' : df_nanmedian,
}

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error
# Define the number of splits (e.g., 5 for 80-20 train-test splits)
n_splits = 3
# Initialize the TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

In [ ]:
import lightgbm as lgb
param = {'verbose' :-1}

model = lgb.LGBMRegressor(**param)
def test(df_dict):
    mean_mae_list = []
    for key in df_dict:
        print(key)
        data = df_dict[key]
        maelist = []
        for train_index, test_index in tscv.split(data):
            train_data = data.iloc[train_index]
            test_data = data.iloc[test_index]

            # Extract target variable for training and testing data
            y_train = train_data['pv_measurement']
            y_test = test_data['pv_measurement']

            # Extract features for training and testing data
            X_train = train_data.drop(columns = 'pv_measurement')
            X_test = test_data.drop(columns = 'pv_measurement')

            # Train the XGBoost model
            model.fit(X_train, y_train,eval_set=[(X_test,y_test)])

            # Make predictions on the test data
            y_pred = model.predict(X_test)

            # Evaluate the model using Mean Absolute Error (MAE)
            mae = mean_absolute_error(y_test, y_pred)
            print("Mean Absolute Error:", mae)
            maelist.append(mae)
        # Return MAE
        mean_mae = np.average(maelist,weights=[1,2,3])
        print("Mean MAE: ",mean_mae )
        mean_mae_list.append(mean_mae)
    return mean_mae_list
test(df_dict)

After this evaluation metrics we found out the iterative imputation and we used this.

In [ ]:
imputer = IterativeImputer(max_iter=5, random_state=42)
for col in X_train_cat.columns:
        X_train_cat[col] = imputer.fit_transform(np.array(X_train_cat[col]).reshape(-1,1))
for col in X_test_cat.columns:
        X_test_cat[col] = imputer.fit_transform(np.array(X_test_cat[col]).reshape(-1,1))

### 3.2. Outliers Detection

The presence of outliers poses significant challenges for certain models. Although we investigated this issue, we ultimately did not address it because we are employing a gradient boosting model, which is less affected by outliers.

#### 3.2.1 IQR for Outliers

In [ ]:
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)

IQR = Q3 - Q1

limite_inferiore = Q1 - 3 * IQR
limite_superiore = Q3 + 3 * IQR

for colonna in X_train.columns:
    X_train[colonna] = X_train[colonna].apply(lambda x: limite_inferiore[colonna] if x < limite_inferiore[colonna] else (limite_superiore[colonna] if x > limite_superiore[colonna] else x))

#### 3.2.2 Z-score

In [ ]:
from scipy import stats
def removal_outliers(df):
    # Sample dataset with outliers
    df_no_outliers = df
    # Calculate Z-scores for each column
    z_scores = np.abs(stats.zscore(df_no_outliers))

    # Define a Z-score threshold (e.g., 2 standard deviations)
    z_score_threshold = 3

    # Create a boolean mask for identifying outliers
    outlier_mask = z_scores > z_score_threshold
    # Define the window size (12 values before and 12 values after)
    window_size = 12
    j =0
    # Replace outliers with the mean of the surrounding values for each column
    for column in df_no_outliers.columns:
        for i in range(len(df_no_outliers)):
            if outlier_mask[column][i]:
                j = j+1
                start = max(i - window_size, 0)
                end = min(i + window_size + 1, len(df_no_outliers))
                df_no_outliers[column][i] = df_no_outliers[column][start:end].mean()
    print(j)
    return df_no_outliers

In [ ]:
df_train_A_no_outliers = removal_outliers(df_train_A)

### 3.3 Scaling

After we moved into scaling the data, but also this was not worth the time spent and the extra computational cost

In [ ]:
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()
X_norm =pd.DataFrame( norm.fit_transform(df[scaling_columns]), columns = df[scaling_columns].columns, index = df.index)
X_norm.describe()

### 3.4 Correlation Analysis

Our strategy was to reduce the dataset's dimensionality by eliminating the most correlated features. However, this approach did not yield the desired results.

#### 3.4.1 Correlation between target and features

In [ ]:
target = 'pv_measurement'

In [ ]:
# Create the dataframe
df = pd.concat([df_train_obs_A, df_train_est_A], ignore_index=True)
# Correlation matrix
correlation_matrix = df.corrwith(df[target])
plt.figure(figsize=(20, 20))
ax = sns.barplot(x=correlation_matrix.values, y=correlation_matrix.index, orient='h')
plt.title('Correlation with pv')
plt.xlabel('Correlation')
plt.ylabel('Feature')
plt.xticks(rotation=90)
for i, v in enumerate(correlation_matrix.values):
    ax.text(v, i, f'{v:.2f}', ha='left', va='center')
plt.show()

#### 3.4.2 Correlation between features

In [ ]:
df = pd.concat([df_train_obs_A, df_train_est_A], ignore_index=True)

# Correlation matrix
correlation_matrix = df.corr()

# Define a threshold
threshold = 0.0

# Find the columns to be kept
columns_to_keep = correlation_matrix.columns[(correlation_matrix >= threshold).sum() >=0]

# Select only those columns satisfying the criterium
filtered_corr_matrix = correlation_matrix.loc[columns_to_keep, columns_to_keep]
mask_upper = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)

# Create a heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f",mask=mask_upper)
plt.title(f'Correlation Heatmap (Threshold = {threshold})')
plt.show()


### 3.5 PairPlot
Following this, we generated a pairplot to examine potential combinations of features that could reveal additional insights.

In [ ]:
sns.pairplot(df)
plt.show()

## 4. Dimension Reduction
Our objective was to apply analytical methods to decrease the dataset's dimensionality, particularly because the existing correlations did not align well with the problem at hand.

#### 4.1. PCA & Pipeline
We incorporated PCA into our pipeline to test its effectiveness in solving the problem. However, this approach also proved unsuccessful. Spoiler alert: none of the dimension reduction techniques we tried worked.

In [ ]:
pipeline = Pipeline(steps=[('Scaler', Normalizer()),
                                 ('PCA', PCA(n_components=40)),
                                 ('Forest', RandomForestRegressor(n_estimators=100, max_depth=5))])

#### 4.2. Feature Selection

##### 4.2.1. Feature importance with Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(scaled_X_train, y_train)
# get importance
importance = rf.feature_importances_

In [ ]:
dataRF = pd.DataFrame()
dataRF['Feature_name'] = scaled_X_train.columns
dataRF['importance'] = importance
dataRF = dataRF.sort_values(by='importance',ascending = True)
plt.figure(figsize=(20, 25))  # width:20, height:3

plot = plt.barh(dataRF['Feature_name'],dataRF['importance'])
plt.bar_label(plot)
plt.rc('ytick', labelsize=8)    # fontsize of the tick labels


In [ ]:
threshold = 0.021

dataRF = dataRF[dataRF['importance'] > threshold]
dataRF['Feature_name'].tolist()

##### 4.2.2. Feature importance with XGBRegressor

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor()
model.fit(scaled_X_train, scaled_Y_train)

  # get importance
importance = model.feature_importances_
dataXGB = pd.DataFrame()
dataXGB['Feature_name'] = scaled_X_train.columns
dataXGB['importance'] = importance
dataXGB = dataXGB.sort_values(by='importance',ascending=True)

In [ ]:

plt.figure(figsize=(20, 25))  # width:20, height:3

plot = plt.barh(dataXGB['Feature_name'],dataXGB['importance'])
plt.bar_label(plot)
plt.rc('ytick', labelsize=8)    # fontsize of the tick labels

In [ ]:
# Specifica il threshold
threshold = 0.01

# Seleziona le righe in cui il valore nella colonna 'Colonna1' supera il threshold
dataXGB = dataXGB[dataXGB['importance'] > threshold]

##### 4.2.3. ExhaustiveFeatureSelector
This took us 30hours of run and was the most useless

In [ ]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector

# IMPORT MODEL TO EVALUATE WITH FEATURES
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
SEED = 42

models = {
    #'rf': RandomForestRegressor(random_state=SEED,),
   # 'lgbm': LGBMRegressor(),
    'xgb': XGBRegressor(random_state=SEED,),
    #'gb': GradientBoostingRegressor(random_state=SEED,),
    #'cb': CatBoostRegressor(random_state=SEED, silent=True),
    #'ab': AdaBoostRegressor(random_state=SEED,),
}

for key in models:
    print(key)
    model = models[key]
    efs = ExhaustiveFeatureSelector(model,
                                   min_features=3,
                                   max_features=5,
                                   scoring='neg_mean_absolute_error',
                                   cv=5,
                                   print_progress=True
                                   )
    efs = efs.fit(X_train, np.ravel(Y_train))

    selected_features = X_train.columns[list(efs.best_idx_)]
    print(selected_features)
    print(efs.best_score_)

## 5 Data Transformation
When scaling the data did not yield results, we shifted our focus to transforming the data to achieve a more Gaussian-like distribution. This decision was influenced by our observation (as noted in section 5.0.1) that most of the columns exhibited right skewness.

### 5.0.1 Histogram

In [ ]:
import math

data = df
num_features = len(data.columns)
num_rows = math.ceil(num_features / 3)

plt.figure(figsize=(12, 8))

for i, column in enumerate(data.columns):
    plt.hist(data[column], bins=20, color='blue', alpha=0.7)
    plt.title(column)
    plt.xlabel("Valore")
    plt.ylabel("Frequenza")
    plt.tight_layout()
    plt.show()

### 5.1. Transformation
We performed this for the train data and the target.

#### 5.1.1 Log Transformation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
original_data = df_train_est_A_imputed['pv_measurement']


# Adding a small constant to avoid division by zero
epsilon = 1e-6
data_with_epsilon = [x + epsilon if x == 0 else x for x in original_data]

# Apply the Box-Cox transformation
transformed_data = np.log(original_data + 1)

# Create a DataFrame to see the original and transformed data side by side
df = pd.DataFrame({'Original Data': original_data, 'Transformed Data': transformed_data})

# Create a figure and two subplots for original and transformed data
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Original vs. Transformed Data')

# Plot the original data
axes[0].hist(original_data, bins=5, edgecolor='black', alpha=0.7, color='blue')
axes[0].set_title('Original Data')
axes[0].set_xlabel('Value')
axes[0].set_ylabel('Frequency')

# Plot the transformed data
axes[1].hist(transformed_data, bins=5, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Transformed Data')
axes[1].set_xlabel('Box-Cox Transformed Value')
axes[1].set_ylabel('Frequency')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
original_data = df_dew_rime['pv_measurement']

# Apply the Box-Cox transformation
transformed_data = np.log(original_data+1)

# Create a DataFrame to see the original and transformed data side by side
df = pd.DataFrame({'Original Data': original_data, 'Transformed Data': transformed_data})

# Create a figure and two subplots for original and transformed data
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Original vs. Transformed Data')

# Plot the original data
axes[0].hist(original_data, bins=5, edgecolor='black', alpha=0.7, color='blue')
axes[0].set_title('Original Data')
axes[0].set_xlabel('Value')
axes[0].set_ylabel('Frequency')

# Plot the transformed data
axes[1].hist(transformed_data, bins=5, edgecolor='black', alpha=0.7, color='green')
axes[1].set_title('Transformed Data')
axes[1].set_xlabel('Box-Cox Transformed Value')
axes[1].set_ylabel('Frequency')

plt.show()

## 6. Maybe all location together?
We attempted to unify all the locations, but encountered several issues, such as encoding the locations and significant differences in feature scales. These challenges led us to abandon this approach and instead continue with the use of three separate models.

### 6.1 One hot encoder
This was our idea to fix the location encoding.

In [ ]:
locations = df_train['Location']
df = pd.get_dummies(df_train, columns=['Location'], prefix=['Location_'])
location_df.index = df_train.index
location_df

In [ ]:
df_with_location = pd.concat([df_train.drop(columns='Location'), location_df], axis=1)

## 7. Feature Engineering
After all this attempt we moved on the feature Engineering and tried to combine the most useful feature, and tried to catch some seasonality with the time information.

### 7.1 Time Feature
We attempted to develop features that could capture the seasonality of the PV output. However, as we discovered through the feature importance analysis in section 4.2, each model required distinct time-related features. Consequently, we created two separate sets of these features to accommodate the varying needs of each model.

In [ ]:

def add_time_features(df, time_column):

    df[time_column] = pd.to_datetime(df[time_column])
    df['hour'] = df[time_column].dt.hour
    df['day_of_week'] = df[time_column].dt.dayofweek
    df['month'] = df[time_column].dt.month
    df['day_of_year'] = df[time_column].dt.dayofyear
    df['week_of_year'] = df[time_column].dt.isocalendar().week
    df['year'] = df[time_column].dt.year

    return df

In [ ]:
def add_time_features_cat(df, time_column):

    df[time_column] = pd.to_datetime(df[time_column])

    df['sin_hour'] = np.sin(np.pi * df[time_column].dt.hour/23.)
    df['sin_month'] = np.sin(np.pi * df[time_column].dt.month/12.)

    return df

In [ ]:
      estimated['time_dummy'] = (estimated['date_forecast'] - estimated['date_forecast'].dt.normalize()).dt.total_seconds() / 3600
      observed['time_dummy'] = 0
      test['time_dummy'] = (test['date_forecast'] - test['date_forecast'].dt.normalize()).dt.total_seconds() / 3600

      estimated['time_delta'] = (estimated['date_calc'] - estimated['date_forecast']).dt.total_seconds() / 3600
      observed['time_delta'] = 0  # since observed data is not forecasting ahead
      test['time_delta'] = (test['date_calc'] - test['date_forecast']).dt.total_seconds() / 3600

### 7.2 Is_estimated feature
These features proved to be highly beneficial, both for us and for the model, in more accurately discerning the differences between the estimated and the observed data.

In [ ]:
      estimated['is_estimated'] = 1
      observed['is_estimated'] = 0
      test['is_estimated'] = 1

### 7.3 Combination between radiation and cloud
These features were pivotal in our project as they provided the model with an insight into the actual solar radiation at any given time, thereby offering a better estimate of the potential PV output.

In [ ]:
 data['weighted_rad'] = ((data['direct_rad:W'] * (1 - data['total_cloud_cover:p']/100)) +
                        (data['diffuse_rad:W'] * (data['total_cloud_cover:p']/100)))


#### 7.4 Atmospheric combination
Recognizing the significant impact of humidity and air density on solar radiation, we attempted to combine these factors to capture more detailed insights.

In [ ]:
data['adjusted_clear_sky_rad'] = (data['clear_sky_rad:W'] *
                                  np.exp(-0.0001 * data['absolute_humidity_2m:gm3']) *
                                  (1 - 0.1 * (data['air_density_2m:kgm3'] - 1.225)))  # Adjusted based on humidity and air density

#### 7.5 Sun Azimuth feature
The idea behind was to captures the cyclical nature.

In [ ]:
    df['sin_sun_azimuth'] = np.sin(np.radians(df['sun_azimuth:d']))

    df['cos_sun_azimuth'] = np.cos(np.radians(df['sun_azimuth:d']))

#### 7.6 Other idea that didn't work out

##### 7.6.1 Sun Zenith

In [ ]:
sun_azimuth = np.array(df['sun_azimuth:d']) * (np.pi / 180)

hour = np.array(df['hour'])
day_of_year = np.array(df['day_of_year'])

solar_time = hour + 0.17 * np.sin(4 * np.pi * (day_of_year - 80) / 373) - 0.129 * np.sin(2 * np.pi * (day_of_year - 8) / 355)
hour_angle = (15 * (12 - solar_time)) * (np.pi / 180)

declination =  0.4093 * np.sin(2 * np.pi * (day_of_year - 81) / 368)

In [ ]:
import math
sun_zenith = np.arccos(
    np.sin(sun_azimuth) * np.sin(declination) + np.cos(sun_azimuth) * np.cos(declination) * np.cos(hour_angle))
sun_zenith_deg = sun_zenith* (180 / np.pi)
plt.plot(sun_zenith_deg[50:100])
plt.show()

##### 7.6.2 Dew or Rime?

In [ ]:
df_dew_rime = df_with_location
df_dew_rime['dew'] = np.where(df_dew_rime['dew_or_rime:idx'] == 1, 1, 0)
df_dew_rime['rime'] = np.where(df_dew_rime['dew_or_rime:idx'] == -1, 1, 0)
df_dew_rime = df_dew_rime.drop(columns = 'dew_or_rime:idx')
df_dew_rime.describe()

# Models
Also here we have run the same model for all the location

## 1. Model Selection
Our model selection was guided by the Kaggle submission scores, which indicated that CatBoost and LightGBM were the most effective. Conversely, using Random Forest or XGBoost did not align well with our approach to the problem.
In this phase, we employed two different approaches to the problem: one involving models that we created and engineered ourselves, and the other utilizing a library known as pyCaret.
We will show only one solution per model, not both. Every model has his own hyperparameter tuning for every location.
We explored two approach for the hypertuning, the first was with optuna and the second with a randomsearchCV.

### 1.1 Catboost

In [ ]:
    # Setup the environment in PyCaret
    exp_reg = setup(data=data, target='target', session_id=seeds,
                    imputation_type="iterative", categorical_iterative_imputer="lightgbm", numeric_iterative_imputer="lightgbm", iterative_imputation_iters = 5,

                    #categorical_features=['dew_or_rime:idx', 'is_in_shadow:idx','is_estimated'],
                    #remove_outliers=True,  #Ble dårligere med denne
                    html=False,
                    experiment_name=f'exp_{location_name}')

    # Create a Catboost model
    cat = create_model('catboost')

    # Tune the model
    tuned_cat = tune_model(cat)

    # Create a bagged version of the tuned model
    bagged_cat = ensemble_model(tuned_cat, method='Bagging')

    # Train on whole dataset
    final_model = finalize_model(bagged_cat)

### 1.2 LightGBM

In [ ]:
def objective(trial):
    param = {
            'metric': 'mae',
            'random_state': 42,
            'n_estimators': trial.suggest_int('n_estimators',10,1000),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4,1e-1),
            'max_depth': trial.suggest_int('max_depth', 1,50),
            'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
            'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100),
            'verbosity': -1
        }
    model = lgb.LGBMRegressor(**param)
    maelist = []
    for train_index, test_index in kfold.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # Extract target variable for training and testing data
        y_train = train_data['pv_measurement']
        y_test = test_data['pv_measurement']

        # Extract features for training and testing data
        X_train = train_data.drop(columns = 'pv_measurement')
        X_test = test_data.drop(columns = 'pv_measurement')

        # Train the XGBoost model
        model.fit(X_train, y_train,eval_set=[(X_test,y_test)])

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Evaluate the model using Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)
        print("Mean Absolute Error:", mae)
        maelist.append(mae)
    # Return MAE
    mean_mae = np.mean(maelist)

    return mean_mae

In [ ]:
import optuna
import warnings
warnings.filterwarnings("ignore")
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
trial = study.best_trial
model_A = lgb.LGBMRegressor(**trial.params)
model_A.fit(X_train,y_train)
df_test_A = df_test_A
y_pred_A = model_A.predict(df_test_A)

### 1.3 Random Forest

In [ ]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
                'n_jobs' : [-1]
             }
print(param_grid)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
rf_Model = RandomForestRegressor()

y_train = train['pv_measurement']
X_train = train.drop(columns = 'pv_measurement')
rf_RandomGrid = RandomizedSearchCV(estimator = rf_Model, param_distributions = param_grid, cv = 10, verbose=2, n_jobs = 4)
rf_RandomGrid.fit(X_train, y_train)

In [ ]:
model_A = RandomForestRegressor(**rf_RandomGrid.best_params_)
model_A.fit(X_train,y_train)
y_pred_A = model_A.predict(test)

### 1.4 XGBoost

In [ ]:
def objective(trial):
    param = {
        'objective':'reg:squarederror',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'gamma': trial.suggest_loguniform('gamma',  1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4,1e-0),
        'n_estimators': trial.suggest_int('n_estimators',10,1000),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'eval_metric':'mae'
    }
    model = xgb.XGBRegressor(**param)
    maelist = []
    for train_index, test_index in tscv.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # Extract target variable for training and testing data
        y_train = train_data['pv_measurement']
        y_test = test_data['pv_measurement']

        # Extract features for training and testing data
        X_train = train_data.drop(columns = 'pv_measurement')
        X_test = test_data.drop(columns = 'pv_measurement')
            # Create an XGBoost DMatrix for training and testing data
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Train the XGBoost model
        model = xgb.train(param, dtrain)

        # Make predictions on the test data
        y_pred = model.predict(dtest)

        # Evaluate the model using Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)
        print("Mean Absolute Error:", mae)
        maelist.append(mae)
    # Return MAE
    mean_mae = np.mean(maelist)

    return mean_mae

### 1.5 Automatic method for all the model

In [ ]:
from scipy.stats import randint, uniform

# Parameters for Random Forest
rf_params = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': uniform(0.01, 0.1),
    'min_samples_leaf': uniform(0.01, 0.1),
    'bootstrap': [True, False]
}

# Parameters for Extra Trees
et_params = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': uniform(0.01, 0.1),
    'min_samples_leaf': uniform(0.01, 0.1),
    'bootstrap': [True, False]
}

# Parameters for XGBoost
xgb_params = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': randint(1, 10)
}

# Parameters for LightGBM
lgbm_params = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(-1, 20),
    'learning_rate': uniform(0.01, 0.3),
    'num_leaves': randint(20, 300),
    'min_child_samples': randint(10, 100),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'verbose' : [-1]
}

# Parameters for CatBoost
cat_params = {
    'iterations': randint(100, 500),
    'depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'l2_leaf_reg': randint(1, 10),
    'border_count': randint(1, 255),
    'subsample': uniform(0.5, 0.5)
}

# Combine all parameter dictionaries into one
model_params = {
    'catboost': cat_params
}

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

def tune_model(X, y, model_params, n_iter=200, cv=2, verbose=1, random_state=42, n_jobs=-1):
    # Store the best estimators here
    best_estimators = {}

    # Define a dictionary with model shorthand and actual regressor objects
    models = {
        'catboost': CatBoostRegressor(verbose=0, thread_count=n_jobs),
    }
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    # Loop through the models and perform Randomized Search
    for name, model in models.items():
        if name in model_params:  # Only if model parameters were provided
            print(f"Tuning hyperparameters for {name}...")
            search = RandomizedSearchCV(
                estimator=model,
                param_distributions=model_params[name],
                n_iter=n_iter,
                cv=cv,
                verbose=verbose,
                random_state=random_state,
                scoring=mae_scorer,
                n_jobs=n_jobs
            )
            search.fit(X, y)
            best_estimators[name] = search.best_estimator_
            print(f"Best parameters for {name}: {search.best_params_}")
        else:
            print(f"No parameters provided for {name}, skipping...")

    return best_estimators

## 2. Single model Plateu
Using a single model, we achieved a plateau with a Kaggle score of 152. At that point, we explored combining models to attain a higher score.

### 2.1 Stacking
Our initial attempt involved stacking the predictions from the LightGBM, Catboost, and RandomForest models, and then applying Linear Regression to this new dataframe. However, this approach did not yield successful results.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Generate some example time series data
# Replace this with your own time series data
data = df

# Define the number of splits for time series cross-validation
n_splits = 3
tscv = TimeSeriesSplit(n_splits=n_splits,test_size = 720)

# Initialize empty arrays to store meta features and true targets
meta_features = np.empty(data.shape[0])

# Iterate through time series cross-validation splits
for train_index, test_index in tscv.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]

    # Split the data into features and target
    X_train = train_data.drop(columns=target)
    y_train = train_data['pv_measurement']
    X_test = test_data.drop(columns=target)
    y_test = test_data['pv_measurement']
    lgbmpar = {'verbose' : -1 }
    lgbm_model = LGBMRegressor(**lgbmpar)
    lgbm_model.fit(X_train, y_train)
    catapar = {'verbose':500}
    # Initialize and train the CatBoost model
    catboost_model = CatBoostRegressor(**catapar)
    catboost_model.fit(X_train, y_train)

    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # Make predictions on the test data
    lgbm_preds = lgbm_model.predict(X_test)
    catboost_preds = catboost_model.predict(X_test)
    rf_preds = rf_model.predict(X_test)
    mae_lgbm = mean_absolute_error(y_test, lgbm_preds)
    mae_cat = mean_absolute_error(y_test, catboost_preds)
    mae_rf = mean_absolute_error(y_test, rf_preds)
    print(f'Lgb : {mae_lgbm}, cat: {mae_cat}, rf: {mae_rf}')
    # Stack the predictions horizontally to create meta features
    stacked_features = np.column_stack((lgbm_preds, catboost_preds,rf_preds))

In [ ]:
# Train the meta-model on the combined feature matrix and the target values
#weightmeans
meta_model = LGBMRegressor()
meta_model.fit(stacked_features, y_test)
Linear_pred = meta_model.predict(stacked_features)

In [ ]:
lgbm_preds = lgbm_model.predict(df_test_A[feature_test])
catboost_preds = catboost_model.predict(df_test_A[feature_test])
rf_preds = rf_model.predict(df_test_A[feature_test])

# Combine the predictions of the base models into a single feature matrix
X_new_meta = np.column_stack((lgbm_preds, catboost_preds,rf_preds))
df_stacked = pd.DataFrame({'lgbm': lgbm_preds, 'cat' :catboost_preds,'rf':rf_preds})
df_stacked['media'] = df_stacked[subset_cols].mean(axis=1)
df_stacked.head(50)
(10 ** df_stacked['lgbm'] -1).plot()

### 2.2 Ensemble modeling
Convinced that our previous solution was missing key details, we decided to increase the complexity of our approach. We developed an ensemble model that utilized the out-of-fold technique, aiming to capture more nuanced aspects of the data.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone

def stacking_ensemble_time_series(models, X, y, meta_learner=ExtraTreesRegressor(), n_folds=5):
    # Create the time series cross-validator
    tscv = TimeSeriesSplit(n_splits=n_folds)

    # To store out-of-fold predictions
    meta_features = np.zeros((y.shape[0], len(models)))

    # Train and generate meta-features
    for idx, (name, model) in enumerate(models.items()):
        print(f"Training base model: {name}")
        oof_predictions = np.zeros(y.shape[0])

        for train_index, test_index in tscv.split(X):
            # Split data into folds
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Clone the model to ensure we have a fresh model
            cloned_model = clone(model)
            cloned_model.fit(X_train, y_train['pv_measurement'])

            # Generate out-of-fold predictions
            oof_predictions[test_index] = cloned_model.predict(X_test)

        # Store out-of-fold predictions as meta-features
        meta_features[:, idx] = oof_predictions

    # Retrieve the last fold for training the meta-learner
    _, meta_train_index = list(tscv.split(X))[-1]
    X_meta_train, y_meta_train = meta_features[meta_train_index, :], y.iloc[meta_train_index]

    # Train the meta-learner on the last fold's meta-features
    print("Training meta-learner...")
    meta_learner.fit(X_meta_train, y_meta_train['pv_measurement'])

    # Fit all the base models on the full training data
    fitted_models = {}
    for name, model in models.items():
        model.fit(X, y['pv_measurement'])
        fitted_models[name] = model

    # Function to make ensemble predictions
    def make_predictions(X_new):
        meta_features_new = np.column_stack([
            fitted_model.predict(X_new) for _, fitted_model in fitted_models.items()
        ])
        return meta_learner.predict(meta_features_new)

    return fitted_models, meta_learner, make_predictions


### 2.3 The mean of different model
In a conversation with a teaching assistant (TA) regarding our project, we discussed hitting a plateau with a single model. The TA suggested averaging the outputs of the same model and perhaps adding another model. This strategy proved to be highly effective; in fact, our best solution was developed based on this idea.
In this solution every model has is own Feature Engineering and Preprocessing so they are like 3 different submission for kaggle where we do the mean

In [ ]:
# LightGBM with some extra features
def process_location_ex(X, y, location_name,seeds):

    # Combine feature data and target into a single DataFrame
    data = X.copy()
    data['target'] = y['pv_measurement']

    exp_reg = setup(data=data, target='target', session_id=seeds,
                    imputation_type="iterative", categorical_iterative_imputer="lightgbm", numeric_iterative_imputer="lightgbm", iterative_imputation_iters = 5,

                    categorical_features=['dew_or_rime:idx', 'is_in_shadow:idx','is_estimated'],
                    html=False,
                    experiment_name=f'exp_{location_name}')

    # Create a LightGBM model
    lightgbm = create_model('lightgbm')

    tuned_lightgbm = tune_model(lightgbm)

    bagged_lightgbm = ensemble_model(tuned_lightgbm, method='Bagging')

    final_model = finalize_model(bagged_lightgbm)


    return final_model

In [ ]:
# Catboost model nr 2
def process_location_cat_2(X, y, location_name,seeds):

    # Dropping some features for this one model
    features_to_drop = ['dew_or_rime:idx', #'snow_density:kgm3',
                        'fresh_snow_3h:cm', 'fresh_snow_1h:cm', 'snow_drift:idx',
                        'snow_depth:cm', 'wind_speed_w_1000hPa:ms', 'prob_rime:p',
                        'fresh_snow_6h:cm', 'snow_melt_10min:mm',
                        'fresh_snow_12h:cm', 'rain_water:kgm2',
                        'super_cooled_liquid_water:kgm2']

    X = X.drop(columns=features_to_drop)

    data = X.copy()
    data['target'] = y['pv_measurement']

    # Setup the environment in PyCaret
    exp_reg = setup(data=data, target='target', session_id=seeds,
                    imputation_type="iterative", categorical_iterative_imputer="lightgbm", numeric_iterative_imputer="lightgbm", iterative_imputation_iters = 5,
                    html=False,
                    experiment_name=f'exp_{location_name}')

    cat = create_model('catboost')

    tuned_cat = tune_model(cat)

    bagged_cat = ensemble_model(tuned_cat, method='Bagging')

    final_model = finalize_model(bagged_cat)


    return final_model

In [ ]:
all_predictions_cat = []
# Catboost model nr 1
for loc in locations:

    # Load your data
    train = pd.read_parquet(f'{loc}/train_targets.parquet').fillna(0)
    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')
    X_train_observed = pd.read_parquet(f'{loc}/X_train_observed.parquet')
    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')

    # Calling preprocessing
    X_train_cat, X_test_cat, is_day_feature1, targets_cat = preprocessing_cat(train, X_train_observed, X_train_estimated, X_test_estimated)

    # Making categorical features
    cat_features = ['dew_or_rime:idx' ,'is_in_shadow:idx']
    X_train_cat['dew_or_rime:idx'] = X_train_cat['dew_or_rime:idx'].astype(int)
    X_train_cat['is_in_shadow:idx'] = X_train_cat['is_in_shadow:idx'].astype(int)
    X_test_cat['dew_or_rime:idx'] = X_test_cat['dew_or_rime:idx'].astype(int)
    X_test_cat['is_in_shadow:idx'] = X_test_cat['is_in_shadow:idx'].astype(int)

    model_cat = CatBoostRegressor(
        loss_function='MAE',
        learning_rate=0.1,
        verbose=200,
        cat_features=cat_features,
        random_state=42)



    # Training
    model_cat.fit(X_train_cat,targets_cat['pv_measurement'])

    # Prediction
    predictions_cat = model_cat.predict(X_test_cat)
    feature_importances = model_cat.get_feature_importance()
    adjusted_final_predictions_cat = predictions_cat * is_day_feature1['is_day:idx']

    adjusted_final_predictions_cat = np.clip(adjusted_final_predictions_cat, 0, None)

    all_predictions_cat.append(adjusted_final_predictions_cat)

# Changing final list to array
all_predictions_cat = np.array(all_predictions_cat)

In [ ]:
all_pred = 0.25*all_predictions_cat+0.25 * all_predictions_lGBM_e+ 0.25* all_predictions_cat_2+0.25*all_predictions_cat_3